# DELTA_DIST+H - Recherche des hyperparamètres d'optimisation

Le notebook précédent recherchant les hyper-paramètres a conduit à des modèles convergeant très rapidement vers une loss de 18, et n'arrivant pas à aller en deça. C'est problématique car cela revient à estimer les distances  de bruit ajouté à 0.018 près en moyenne, alors qu'on souhaiterait obtenir une précision à 0.001 pour pouvoir optimiser des molécules de façon suffisamment efficace.

Nous allons ici redéfinir des fonctions similaires au notebook précédent, mais en faisant varier les hyperparamètres propres à l'optimisation (choix de l'optimiseur, paramètres de l'optimiseur). De plus, nous allons entraîner les modèles sur seulement un million d'exemples, car cela suffira pour voir si l'optimisation converge vite ou non, et jusqu'à quelle loss si oui.

In [4]:
minimal_prepared_input_loc = "../data/minimal_set_riken_v2_prepared_input.h5"
minimal_labels_loc = "../data/minimal_set_riken_v2_labels.h5"

train_prepared_input_loc = "../data/train_set_riken_v2_prepared_input.h5"
train_labels_loc = "../data/train_set_riken_v2_labels.h5"

models_loc = "../models/DELTA_DIST+H_01/4.1.QUADRI_OPTI/models/"
logs_loc = "../models/DELTA_DIST+H_01/4.1.QUADRI_OPTI/logs/"

## Création du RN

### Fonctions de coût et d'évaluation du modèle

#### Fonction renvoyant le masque des atomes à prendre en compte pour les calculs

L'entrée et la sortie du RN étant définies par une méthode de padding, seul un certain nombre d'entrées et de sortie est utilisé pour chaque exemple d'entraînement en fonction du nombre d'atomes de la molécule. On définit ici une fonction qui renvoie le masque des différences de distances à prendre en compte sur les données en entrée et les étiquettes.

In [5]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


#### Fonction de coût

In [6]:
def partial_rmse(predictions, targets):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    
    with tf.name_scope("partial_rmse"):

        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)
        
        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)   

        return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Fonction d'évaluation des performances (score R2)

In [7]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    with tf.name_scope("partial_r2"):
    
        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)

        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)

        # Calcul de l'erreur totale
        total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets_masked))))

        # Calcul de l'erreur inexpliquée
        unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets_masked, predictions_masked)))

        r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")
        return r2

## Création du RN

### Fonctions de coût et d'évaluation du modèle

Les fonctions ci-dessous sont très fortement inspirées du travail de Nicolas Roux lors de son TER de M1 en 2017. Les différences sont les suivantes.

* Calcul du RMSE uniquement sur les atomes définis
* Utilisation d'un score R2 pour la validation
* Recherche par quadrillage des hyper-paramètres

#### Fonction renvoyant le masque des atomes à prendre en compte pour les calculs

L'entrée et la sortie du RN étant définies par une méthode de padding, seul un certain nombre d'entrées et de sortie est utilisé pour chaque exemple d'entraînement en fonction du nombre d'atomes de la molécule. On définit ici une fonction qui renvoie le masque des différences de distances à prendre en compte sur les données en entrée et les étiquettes.

In [8]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


#### Fonction de coût

In [9]:
def partial_rmse(predictions, targets):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    
    with tf.name_scope("partial_rmse"):

        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)
        
        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)   

        return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Fonction d'évaluation des performances (score R2)

In [10]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    with tf.name_scope("partial_r2"):
    
        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)

        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)

        # Calcul de l'erreur totale
        total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets_masked))))

        # Calcul de l'erreur inexpliquée
        unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets_masked, predictions_masked)))

        r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")
        return r2

#### Définition d'une fonction créant le RN

In [29]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam


def creer_1k_4x8650_800(epsilon, learning_rate):
    """ Fonction créant un réseau de neurones de type fully connected, ayant une couche d'entrée de 1000
    neurones, quatre couches cachées de 8650 neurones et une sortie de 800 neurones
    Inputs : hyperparamètres de l'optimizer adam
    """
    do = 0.99
    stddev_init = 0.001

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 1000], name='input')

    
    # On créé les trois couches cachées
    for i in range(4):
        network = fully_connected(network, 8650, activation='relu', name='fc'+str(i), weights_init=winit)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, do)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 800, activation='prelu', name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=partial_rmse, metric=partial_r2_score, name='target')
            
    return network


## Préparation des données

#### Fonction renvoyant deux sous-ensembles du jeu d'entrainement : un ensemble d'exemples et les cibles correspondantes

In [30]:
def get_fold(train_set, targets, reduce_train_fold_size):
    """ Permet d'obtenir un sous-ensemble du jeu d'entraînement afin de ne pas travailler sur le jeu
    d'entraînement total pour la recherche par quadrillage et donc de gagner du temps d'exécution. L'idée
    et que si un ensemble d'hyper-paramètres produit des meilleurs résultats que les autres ensembles
    d'hyper-paramètres sur l'ensemble du jeu d'entraînement, alors on suppose que ce sera également 
    le cas sur une partie des données. """

    return (train_set["inputs"][:reduce_train_fold_size], targets["targets"][:reduce_train_fold_size])
    
    

## Entraînement des modèles

#### Fonction d'entraînement de modèles pour recherche des meilleurs hyperparamètres

In [31]:
import h5py
import tflearn as tfl
import time
from scipy import sparse
import numpy as np
import gc
import tensorflow as tf


def train(input_path, targets_path, fold_size, samples_per_batch, epochs):
    
    
    learning_rate_vals = [0.001, 0.0001]
    epsilon_vals = [0.1, 1e-8]

    total_start_time = time.time()
    
    inputs_h5 = h5py.File(input_path, 'r')
    targets_h5 = h5py.File(targets_path, 'r')
    
    input_X, labels_y = get_fold(inputs_h5, targets_h5, fold_size)

    
    # On itère sur toutes les combinaisons d'hyperparamètres
    for learning_rate in learning_rate_vals:
        for epsilon in epsilon_vals:
                                
            tf.reset_default_graph()

            start_time = time.time()

            nom_modele = "DELTA_DIST+H_01_hyperparam_lr-"+str(learning_rate)+"_epsilon_"+str(epsilon)

            print()
            print("Training model "+nom_modele)
        
            # On créé le réseau avec les hyperparamètres courants
            network = creer_1k_4x8650_800(learning_rate, epsilon)

            # On créé le modèle
            model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_loc)

            print("Début entraînement")

            # Entraînement
            model.fit(X_inputs=input_X,Y_targets=labels_y, batch_size=samples_per_batch,
                      shuffle = True, snapshot_step=100, validation_set=0.1,
                      show_metric=True, run_id="hyperparametres_"+nom_modele , n_epoch=epochs)

            model.save(models_loc+"test_" + nom_modele)

            print("Temps d'entraînement du modèle : ")
            print("--- %s seconds ---" % (time.time() - start_time))

            inputs_h5.close()
            targets_h5.close()
    
    
    print("Temps total :")
    print("--- %s seconds ---" % (time.time() - total_start_time))



In [32]:
# On fait la recherche par quadrillage sur deux millions et demi d'exemples avec une taille de batch de 1000 
# et une seule époque d'entraînement par modèle
train(train_prepared_input_loc, train_labels_loc, 2500000, 1000, 1)
#train(minimal_prepared_input_loc, minimal_labels_loc, 100000, 1000, 2)


Training model DELTA_DIST+H_01_hyperparam_lr-0.001_epsilon_0.1


/home/etudiant/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


ResourceExhaustedError: OOM when allocating tensor of shape [800] and type float
	 [[Node: outlayer/b/Adam/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [800] values: 0 0 0...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'outlayer/b/Adam/Initializer/zeros', defined at:
  File "/home/etudiant/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/etudiant/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/etudiant/anaconda3/lib/pyt